# Goal: Find error w/ "affiliations," "parent count," and "departmental count"
- Probably regex error
- Problem: Abstract is truncated --> Solved, issue with indexing --> Implemented
- Problem: Different Funding stated within the article --> Soln?: scrape article directly
- Problem: Country --> Soln: They dont know what country codes are
- Problem: Parent Orgs --> only scraped from top --> implemented

In [1]:
import pandas as pd

In [2]:
df_error_data = pd.read_csv("../final_data.csv").fillna("")
df_error_data.head()

,Unnamed: 0,Title,PMID,Authors,Affiliations,Abstract,Journal,Grants_and_Funding,Conflict_of_Interest,Impact_Factor,...,PUB_TYPES,ORGS_AND_LOCS,raw_ner,DEPTS,DEPT_COUNT,ABSTRACT_CATEGORIES,KIDNEY,TRANSPLANT,BOTH,DEPT_SILO
0,1,Long-Term Outcomes in IgA Nephropathy.,37055195,"David Pitcher,Fiona Braddon,Bruce Hendry,Alex ...","UK Renal Registry, The UK Kidney Association, ...",IgA nephropathy can progress to kidney failure...,Clinical journal of the American Society of Ne...,MR/R013942/1||MRC_||Medical Research Council||...,J. Barratt reports consultancy for Alnylam Pha...,9.8,...,{'Journal Article'},"{'ORGS': {'JAMCO Pharma Consulting', 'Universi...","{'Bristol Royal Hospital for Children', 'Londo...",{'Department of Renal Medicine'},1,Clinical Research in Nephrology,Yes,No,No,Yes
1,5,Treatment Effect of the SGLT2 Inhibitor Empagl...,36396331,"Julie Refardt,Cornelia Imber,Rianne Nobbenhuis...","Department of Endocrinology, Diabetes, and Met...",The syndrome of inappropriate antidiuresis (SI...,Journal of the American Society of Nephrology ...,,All authors have nothing to disclose.,13.6,...,"{'Journal Article', 'Randomized Controlled Tri...","{'ORGS': {'Felix Platter Hospital', 'Departmen...","{'Department of Endocrinology, Diabetes, and M...",{'University Department of Geriatric Medicine'...,4,Clinical Research in Nephrology,Yes,No,No,No
2,6,Treatment of IgA Nephropathy: A Rapidly Evolvi...,37772889,"Khalil El Karoui,Fernando C Fervenza,An S De V...","Department of Nephrology, H√¥pital Tenon, Sorb...",The pivotal event in the pathophysiology of Ig...,Journal of the American Society of Nephrology ...,,A.S. De Vriese is scientific advisor/consultan...,13.6,...,{'Journal Article'},"{'ORGS': {'Mayo Clinic', 'Sorbonne Universit√©...","{'Division of Nephrology and Hypertension', 'S...","{'Division of Nephrology and Hypertension', 'D...",3,Clinical Research in Nephrology,Yes,No,No,No
3,7,Management of Poisonings and Intoxications.,37097121,"Marc Ghannoum,Darren M Roberts","Research Center, CIUSSS du Nord-de-l'√Æle-de-M...",Poisoning occurs after exposure to any of a nu...,Clinical journal of the American Society of Ne...,,,9.8,...,{'Journal Article'},"{'ORGS': {""Sydney Children's Hospitals Network...","{'Sydney', 'New South Wales', 'Research Center...",{'Department of Nephrology and Hypertension'},1,Blood Purification,Yes,No,No,Yes
4,13,Clinical and Prognostic Factors in Patients wi...,37283461,"Anis Chaba,Arnaud Devresse,Vincent Audard,Jean...",Departement of Nephrology-Dialysis-Transplanta...,IgG4-related kidney disease is a major manifes...,Clinical journal of the American Society of Ne...,,"V. Audard reports consultancy for, honoraria f...",9.8,...,"{'Journal Article', 'Observational Study'}","{'ORGS': {'Clinique Sainte Anne', 'CHU Rennes'...",{'Departement of Nephrology-Dialysis-Transplan...,{'Service de M√©decine Interne et Immunologie ...,4,Clinical Research in Nephrology,Yes,No,No,No


In [3]:
PMID_TO_CHECK = 37055195

In [4]:
df_error_data.columns

Index(['Unnamed: 0', 'Title', 'PMID', 'Authors', 'Affiliations', 'Abstract',
       'Journal', 'Grants_and_Funding', 'Conflict_of_Interest',
       'Impact_Factor', 'Mesh Terms', 'MeSH Term Descriptor IDs',
       'Total Affiliations', 'Total Authors', 'Pub Type', 'PARENT_ORGS',
       'COUNTRIES', 'COUNTRY_COUNT', 'PARENT_COUNT', 'SILO', 'FUNDING_TYPE',
       'COI?', 'PUB_TYPES', 'ORGS_AND_LOCS', 'raw_ner', 'DEPTS', 'DEPT_COUNT',
       'ABSTRACT_CATEGORIES', 'KIDNEY', 'TRANSPLANT', 'BOTH', 'DEPT_SILO'],
      dtype='object')

In [5]:
df_error_data[df_error_data.PMID == PMID_TO_CHECK]['PARENT_COUNT'][0]

7

In [6]:
"David Pitcher 1 2, Fiona Braddon 1, Bruce Hendry 3, Alex Mercer 4, Kate Osmaston 1, Moin A Saleem 5, Retha Steenkamp 1, Katie Wong 1 2, A Neil Turner 6, Kaijun Wang 3, Daniel P Gale 2, Jonathan Barratt 7".split(',')

['David Pitcher\xa01\xa02',
 '\xa0Fiona Braddon\xa01',
 '\xa0Bruce Hendry\xa03',
 '\xa0Alex Mercer\xa04',
 '\xa0Kate Osmaston\xa01',
 '\xa0Moin A Saleem\xa05',
 '\xa0Retha Steenkamp\xa01',
 '\xa0Katie Wong\xa01\xa02',
 '\xa0A Neil Turner\xa06',
 '\xa0Kaijun Wang\xa03',
 '\xa0Daniel P Gale\xa02',
 '\xa0Jonathan Barratt\xa07']

In [7]:
from Bio import Entrez
import pandas as pd
from silo_utils_TESTING import *

#TESTING COPY
# Rewrite of silo_transplant by Timur Seckin to leverage the Entrez api for better reproducibilty
#"""
# Original URL: "https://pubmed.ncbi.nlm.nih.gov/?term=%28Journal+of+the+American+Society+of+Nephrology%5BJournal%5D+OR+American+Journal+of+Kidney+Diseases%5BJournal%5D+OR+Clinical+Journal+of+the+American+Society+of+Nephrology%5BJournal%5D+OR+American+Journal+of+Transplantation%5BJournal%5D+OR+Transplantation%5BJournal%5D+OR+Nephrology+Dialysis+Transplantation%5BJournal%5D%29+AND+%28clinical+research+OR+original+research+OR+basic+research+OR+clinical+investigation+OR+clinical+epidemiology+OR+original+investigation+OR+original+article+OR+original+basic+science+OR+original+clinical+science%29+AND+2023%5BPublication+Date%5D&size=200"
#"""

#silo_query = "(Journal of the American Society of Nephrology[Journal] OR American Journal of Kidney Diseases[Journal] OR Clinical Journal of the American Society of Nephrology[Journal] OR American Journal of Transplantation[Journal] OR Transplantation[Journal] OR Nephrology Dialysis Transplantation[Journal]) AND (clinical research OR original research OR basic research OR clinical investigation OR clinical epidemiology OR original investigation OR original article OR original basic science OR original clinical science) AND 2023[Publication Date]"
# Kidney International?
#ret_num = 622
#probe = search(silo_query,ret_num)

#for article_id in probe['IdList']:
#    print("Scraping article: ", article_id)
#    scraped_article_list.append(article_scraper(retrieve_data(article_id)))

# df = pd.DataFrame(scraped_article_list)
# df.to_csv("full.csv", index=False)
# print("Data extraction complete. Check full.csv for results.")


StreamModeError: the XML file must be opened in binary mode.

In [10]:


pmid = str(PMID_TO_CHECK)

handle = Entrez.elink(dbfrom="pubmed", id=pmid)

record = Entrez.read(handle)

handle.close()

record

[{'ERROR': [], 'LinkSetDb': [{'Link': [{'Id': '37055195'}, {'Id': '24408121'}, {'Id': '21965586'}, {'Id': '27557557'}, {'Id': '37931634'}, {'Id': '33514642'}, {'Id': '28763548'}, {'Id': '39248402'}, {'Id': '31907076'}, {'Id': '32162319'}, {'Id': '33561333'}, {'Id': '19628647'}, {'Id': '35579642'}, {'Id': '32990213'}, {'Id': '38364955'}, {'Id': '37429271'}, {'Id': '37591292'}, {'Id': '29657199'}, {'Id': '28592423'}, {'Id': '32507961'}, {'Id': '37698655'}, {'Id': '38261310'}, {'Id': '32642919'}, {'Id': '26228193'}, {'Id': '36745456'}, {'Id': '32694247'}, {'Id': '31517550'}, {'Id': '35659570'}, {'Id': '21663387'}, {'Id': '29529608'}, {'Id': '25003873'}, {'Id': '20470285'}, {'Id': '30420158'}, {'Id': '37816675'}, {'Id': '31605314'}, {'Id': '30099439'}, {'Id': '21737517'}, {'Id': '28233100'}, {'Id': '25134778'}, {'Id': '28130969'}, {'Id': '33775708'}, {'Id': '38287343'}, {'Id': '37871587'}, {'Id': '37336802'}, {'Id': '24578331'}, {'Id': '37314777'}, {'Id': '37062274'}, {'Id': '28956741'}, {

In [80]:
test_article['PubmedArticle'][0]['MedlineCitation']


DictElement({'SpaceFlightMission': [], 'OtherID': [], 'GeneralNote': [], 'InvestigatorList': [], 'OtherAbstract': [], 'KeywordList': [], 'CitationSubset': ['IM'], 'PMID': StringElement('37055195', attributes={'Version': '1'}), 'DateCompleted': {'Year': '2023', 'Month': '06', 'Day': '12'}, 'DateRevised': {'Year': '2024', 'Month': '06', 'Day': '03'}, 'Article': DictElement({'ELocationID': [StringElement('10.2215/CJN.0000000000000135', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})], 'ArticleDate': [DictElement({'Year': '2023', 'Month': '04', 'Day': '13'}, attributes={'DateType': 'Electronic'})], 'Language': ['eng'], 'Journal': {'ISSN': StringElement('1555-905X', attributes={'IssnType': 'Electronic'}), 'JournalIssue': DictElement({'Volume': '18', 'Issue': '6', 'PubDate': {'Year': '2023', 'Month': 'Jun', 'Day': '01'}}, attributes={'CitedMedium': 'Internet'}), 'Title': 'Clinical journal of the American Society of Nephrology : CJASN', 'ISOAbbreviation': 'Clin J Am Soc Nephrol'}, 'ArticleTitl

In [72]:
test_article['PubmedArticle'][0]['MedlineCitation'].keys()




#['Article'].keys()

dict_keys(['SpaceFlightMission', 'OtherID', 'GeneralNote', 'InvestigatorList', 'OtherAbstract', 'KeywordList', 'CitationSubset', 'PMID', 'DateCompleted', 'DateRevised', 'Article', 'MedlineJournalInfo', 'CommentsCorrectionsList', 'MeshHeadingList', 'CoiStatement'])

In [ ]:
pd.read